In [46]:
from bs4 import BeautifulSoup
import requests
import random
import pandas as pd
import re
import time
import numpy as np

In [2]:
user_agent = {'User-agent': 'Mozilla/5.0'}

In [25]:
artists=[]
years=[]
release_countries=[]
labels=[]
artist_links=[]
albums=[]
album_links=[]

In [26]:
for i in range(0,21):
    '''grabs most collected records from discogs'''
    if i==0:
        url = "https://www.discogs.com/search/?sort=have%2Cdesc&ev=em_rs&format_exact=Vinyl&layout=sm"
    if i > 0:
        url = "https://www.discogs.com/search/?sort=have%2Cdesc&ev=em_rs&format_exact=Vinyl&layout=sm&page=" + str(i)
    time.sleep(30+5*random.random())
    response = requests.get(url, headers = user_agent)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    #adding artists to the lists
    for element in soup.find(id='search_results').find_all(class_="card_body"):
        try:
            years.append(element.find_all(class_="card_release_year")[0].text)
        except:
            years.append(np.nan)
        try:
            labels.append(element.find('p', class_="card_info").find_all('a')[0].text)
        except:
            labels.append(np.nan)
        try:
            release_countries.append(element.find_all(class_="card_release_country")[0].text)
        except:
            release_countries.append(np.nan)
        try:
            albums.append(element.find_all("a", class_="search_result_title")[0].text)
        except:
            albums.append(np.nan)
        try:
            album_links.append('https://www.discogs.com'+element.find_all('a', class_="search_result_title")[0].get('href'))
        except:
            album_links.append(np.nan)
        try:
            artists.append(element.find_all('a')[0].text)
        except:
            artists.append(np.nan)
        try:
            artist_links.append('https://www.discogs.com'+element.find_all('a')[0].get('href'))
        except:
            artist_links.append(np.nan)

In [27]:
len(years)

1050

In [28]:
len(albums)

1050

In [29]:
len(artists)

1050

In [30]:
len(artist_links)

1050

In [31]:
len(album_links)

1050

In [32]:
len(labels)

1050

In [33]:
len(release_countries)

1050

In [20]:
top_collected = pd.DataFrame({'artists':artists,'albums':albums, 'artist_links':artist_links, 'album_links':album_links, 'release_year':years, 'label':labels, 'release_country':release_countries})

In [34]:
top_all_collected = pd.DataFrame({'artists':artists,'albums':albums, 'artist_links':artist_links, 'album_links':album_links, 'release_year':years, 'label':labels, 'release_country':release_countries})

In [35]:
top_all_collected.tail()

,artists,albums,artist_links,album_links,release_year,label,release_country
1045,The Beatles,The Beatles At The Hollywood Bowl,https://www.discogs.com/artist/82730-The-Beatles,https://www.discogs.com/The-Beatles-The-Beatle...,1977,Parlophone,UK
1046,Fine Young Cannibals,The Raw & The Cooked,https://www.discogs.com/artist/4711-Fine-Young...,https://www.discogs.com/Fine-Young-Cannibals-T...,1988,London Records,UK
1047,Elton John,A Single Man,https://www.discogs.com/artist/57103-Elton-John,https://www.discogs.com/Elton-John-A-Single-Ma...,1978,The Rocket Record Company,UK
1048,David Bowie,Never Let Me Down,https://www.discogs.com/artist/10263-David-Bowie,https://www.discogs.com/David-Bowie-Never-Let-...,1987,EMI America,UK
1049,Daryl Hall + John Oates,H2O,https://www.discogs.com/artist/95886-Daryl-Hal...,https://www.discogs.com/Daryl-Hall-John-Oates-...,1982,RCA,US


In [58]:
top_all_collected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   artists          1050 non-null   object
 1   albums           1050 non-null   object
 2   artist_links     1050 non-null   object
 3   album_links      1050 non-null   object
 4   release_year     1047 non-null   object
 5   label            1050 non-null   object
 6   release_country  1049 non-null   object
dtypes: object(7)
memory usage: 57.5+ KB


In [36]:
import pickle

In [38]:
with open('top_1050_collected_records.pickle', 'wb') as save_file:
    pickle.dump(top_all_collected, save_file)

In [39]:
import os
os.getcwd()

'/Users/beth/Desktop'

In [42]:
from datetime import datetime

In [49]:
first_release_links = []
number_of_songs= []
average_song_length= []

In [50]:
for url in top_all_collected['album_links']:
    #from album main page
    time.sleep(10+2*random.random())
    response = requests.get(url, headers = user_agent)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    #link to album 1st release page
    try:
        first_release_links.append('https://www.discogs.com' + soup.find('table', id='versions').find('td', class_='title').find_all('a')[0].get('href'))
    except:
        first_release_links.append(np.nan)
    
    #number of songs
    s=0
    try:
        for element in soup.find('table', class_="playlist").find_all('tr', class_="tracklist_track track"):
            for song in element.find('span', class_="tracklist_track_title"):
                s += 1
        number_of_songs.append(s)
    except:
        number_of_songs.append(np.nan)
    
    #avg song length
    form='%M:%S'
    song_lengths=[]
    try:
        for element in soup.find('table', class_="playlist").find_all('tr', class_="tracklist_track track"):
            t = element.find('td', class_="tracklist_track_duration").find('span').text
            t = datetime.strptime(t,form)
            song_lengths.append(t)
        avg_time=datetime.strftime(datetime.fromtimestamp(sum(map(datetime.timestamp,song_lengths))/len(song_lengths)),"%M:%S")
        average_song_length.append(avg_time)
    except:
        average_song_length.append(np.nan)
        
        

In [51]:
album_main_pages = pd.DataFrame({'first_release_links':first_release_links,'number_of_songs':number_of_songs,'average_song_length':average_song_length})

In [56]:
album_main_pages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   first_release_links  1050 non-null   object
 1   number_of_songs      1050 non-null   int64 
 2   average_song_length  831 non-null    object
dtypes: int64(1), object(2)
memory usage: 24.7+ KB


In [54]:
with open('album_main_pages.pickle', 'wb') as save_file:
    pickle.dump(album_main_pages, save_file)

In [ ]:
total_artist_albums = []
artist_first_years = []
artist_last_years = []

In [ ]:
for url in top_all_collected['artist_links']:
    #get info from artist pages
    response = requests.get(url, headers = user_agent)
    time.sleep(10+2*random.random())
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    #total albums
    try:
        total_artist_albums.append(soup.find(id="discography_wrapper").find_all('span', class_='facet_count')[1].text)
    except:
        total_artist_albums.append(np.nan)
        
    #first album year
    try:
        artist_first_years.append(soup_artist.find('table', id="artist").find_all('td', class_="year has_header")[0].text)
    except:
        artist_first_years.append(np.nan)
        
    #last album year
    try:
        url=url + "?sort=year%2Cdesc&limit=25"
        time.sleep(5+1*random.random())
        response = requests.get(url, headers = user_agent)
        artist_last_years.append(soup_artist.find('table', id="artist").find_all('td', class_="year has_header")[0].text)
    except:
        artist_last_years.append(np.nan)
    

In [ ]:
users_have = []
users_want = []
user_rating = []
lowest_price = []
median_price = []
highest_price = []
last_sold = []
number_for_sale = []
styles = []
genres = []

In [ ]:
for url in top_all_collected['first_release_links']:
    #get info from the first release of an album page
    response = requests.get(url, headers = user_agent)
    time.sleep(10+2*random.random())
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    
    #getting info for the record stats, including price
    stats =[element for element in soup_album.find(id = "statistics").find_all('li')]
    try:
        users_have.append(re.search(':\n([0-9]*)',stats[0].text).group(1))
    except:
        users_have.append(np.nan)
    try:
        users_want.append(re.search(':\n([0-9]*)',stats[1].text).group(1))
    except:
        users_want.append(np.nan)
    try:
        user_rating.append(re.search(':\n([0-9].[0-9]{1,2})', stats[2].text).group(1))
    except:
        user_rating.append(np.nan)
    try:
        lowest_price.append(re.search('\$([0-9]*.[0-9]{2})',stats[5].text).group(1))
    except:
        lowest_price.append(np.nan)
    try:
        median_price.append(re.search('\$([0-9]*.[0-9]{2})',stats[6].text).group(1))
    except:
        median_price.append(np.nan)
    try:
        highest_price.append(re.search('\$([0-9]*.[0-9]{2})',stats[7].text).group(1))
    except:
        highest_price.append(np.nan)
    try:
        last_sold.append(re.search(':\n([0-9]{2} [A-z]{3} [0-9]{2})',stats[4].text).group(1))
    except:
        last_sold.append(np.nan)
        
    #getting for sale info from marketplace section
    try:
        element = soup_album.find(class_ = "marketplace_for_sale_count").find('strong').text
        number_for_sale.append(re.search('([0-9,]*)',element).group())
    except:
        number_for_sale.append(np.nan)
        
    #find genre and styles (may be multiple, so a list)
    try:
        for link in soup_album.find(class_="profile").find_all("a"):
            all_styles=[]
            if 'style' in link.get('href'):
                all_styles.append(link.text)
        style.append(all_styles)
    except:
        style.append(np.nan)
        
    try:
        for link in soup_album.find(class_="profile").find_all("a"):
            all_genres=[]
            if 'genre' in link.get('href'):
                all_genres.append(link.text)
        genre.append(all_genres)
    except:
        genre.append(np.nan)
    